### Load Library ###

In [1]:
import os
from pathlib import Path
import numpy as np
import yaml
from tqdm import tqdm
import torch
import sys
sys.path.append('../')

from models.models import *
from utils.datasets import create_dataloader
from utils.general import non_max_suppression, increment_path, check_dataset, check_img_size
from utils.torch_utils import select_device
from utils.metrics import ap_per_class, compute_ap
from feature_mining.utils import *

In [2]:
# Set parameters
%matplotlib inline
imgsz=1600
imgsz_test=1600
# img_size = (1600, 900)
batch_size=4
gs=64
cache_image=True
rect=True
pad=0.5
conf_thres=0.001
iou_thres=0.6
cfg = '../cfg/yolov4-tiny-25.cfg'
device_str = 'cpu'
weights='../weights/best.pt'
data='../data/nuimages.yaml'
save_dir = Path(increment_path(Path('runs/visualize') / 'yolov4-tiny-25', exist_ok=True))  # increment run
save_dir.mkdir(parents=True, exist_ok=True)

### Set Up Dataset ###
Download nuScenes dataset and uncompress it.

In [3]:
# Initialization
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from nuimages import NuImages

# Load NuImages
nuim_train = NuImages(dataroot='/root/data/nuimages/images/', version='v1.0-train', verbose=True, lazy=False)
nuim_val = NuImages(dataroot='/root/data/nuimages/images/', version='v1.0-val', verbose=True, lazy=False)

Loading nuImages tables for version v1.0-train...
Loaded 12 attribute(s) in 0.019s,
Loaded 2022 calibrated_sensor(s) in 0.025s,
Loaded 25 category(s) in 0.000s,
Loaded 872181 ego_pose(s) in 7.088s,
Loaded 350 log(s) in 0.007s,
Loaded 557715 object_ann(s) in 4.603s,
Loaded 67279 sample(s) in 0.144s,
Loaded 872181 sample_data(s) in 4.679s,
Loaded 6 sensor(s) in 0.000s,
Loaded 73755 surface_ann(s) in 3.030s,
Done loading in 19.596 seconds (lazy=False).
Loading nuImages tables for version v1.0-val...
Loaded 12 attribute(s) in 0.000s,
Loaded 464 calibrated_sensor(s) in 0.011s,
Loaded 25 category(s) in 0.000s,
Loaded 213185 ego_pose(s) in 2.013s,
Loaded 82 log(s) in 0.014s,
Loaded 136074 object_ann(s) in 0.886s,
Loaded 16445 sample(s) in 0.052s,
Loaded 213185 sample_data(s) in 1.168s,
Loaded 6 sensor(s) in 0.021s,
Loaded 17995 surface_ann(s) in 0.777s,
Done loading in 4.944 seconds (lazy=False).


In [4]:
from models import models

# device = select_device(device_str)
device = 'cuda:2'

# Load with checkpoint
model = Darknet(cfg).to(device)  # create model

ckpt = torch.load(weights, map_location=device)  # load checkpoint
ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].numel() == v.numel()}
model.load_state_dict(ckpt['model'], strict=False)

# Evaluation mode
model.eval()

Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (Conv2d): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(32, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (Conv2d): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (Conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm2d): BatchNorm2d(64, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): FeatureConcat_l()
    (4): Sequential(
      (Conv2d): Conv2d(32, 32, kernel_size=(3, 3), 

In [5]:
imgsz = check_img_size(imgsz, s=64)
imgsz_test = check_img_size(imgsz_test, s=64)

with open(data) as f:
    data_dict = yaml.load(f, Loader=yaml.FullLoader)  # data dict

check_dataset(data_dict)  # check    
train_path = data_dict['train']
val_path = data_dict['val']
nc, names = (int(data_dict['nc']), data_dict['names'])  # number classes, names
assert len(names) == nc, '%g names found for nc=%g dataset in %s' % (len(names), nc, opt.data)  # check
print(names)

['animal', 'flat.driveable_surface', 'human.pedestrian.adult', 'human.pedestrian.child', 'human.pedestrian.construction_worker', 'human.pedestrian.personal_mobility', 'human.pedestrian.police_officer', 'human.pedestrian.stroller', 'human.pedestrian.wheelchair', 'movable_object.barrier', 'movable_object.debris', 'movable_object.pushable_pullable', 'movable_object.trafficcone', 'static_object.bicycle_rack', 'vehicle.bicycle', 'vehicle.bus.bendy', 'vehicle.bus.rigid', 'vehicle.car', 'vehicle.construction', 'vehicle.ego', 'vehicle.emergency.ambulance', 'vehicle.emergency.police', 'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck']


In [ ]:
trainloader, trainset = create_dataloader(train_path, imgsz, batch_size, gs)
mlc = np.concatenate(trainset.labels, 0)[:, 0].max()  # max label class
nb = len(trainloader)  # number of batches
assert mlc < nc, 'Label class %g exceeds nc=%g in %s. Possible class labels are 0-%g' % (mlc, nc, data, nc - 1)

In [6]:
valloader, valset = create_dataloader(val_path, imgsz, batch_size, gs)
mlc = np.concatenate(valset.labels, 0)[:, 0].max()  # max label class
nb = len(valloader)  # number of batches
assert mlc < nc, 'Label class %g exceeds nc=%g in %s. Possible class labels are 0-%g' % (mlc, nc, data, nc - 1)

Scanning labels /root/data/nuimages/labels/samples/CAM_BACK.cache3 (14873 found, 0 missing, 1572 empty, 0 duplicate, for 16445 images): 16445it [00:01, 13497.35it/s]


In [123]:
names = ['animal', 'flat.driveable_surface', 'human.pedestrian.adult', 'human.pedestrian.child', 
        'human.pedestrian.construction_worker', 'human.pedestrian.personal_mobility', 'human.pedestrian.police_officer', 
        'human.pedestrian.stroller', 'human.pedestrian.wheelchair', 'movable_object.barrier', 'movable_object.debris', 
        'movable_object.pushable_pullable', 'movable_object.trafficcone', 'static_object.bicycle_rack', 'vehicle.bicycle', 
        'vehicle.bus.bendy', 'vehicle.bus.rigid', 'vehicle.car', 'vehicle.construction', 'vehicle.ego', 
        'vehicle.emergency.ambulance', 'vehicle.emergency.police', 'vehicle.motorcycle', 'vehicle.trailer', 'vehicle.truck']
attribute_names = ['cycle.with_rider', 'cycle.without_rider', 'pedestrian.moving', 'pedestrian.sitting_lying', 'pedestrian.standing',
        'vehicle.moving', 'vehicle.parked', 'vehicle.stopped', 'vehicle_light.emergency.flashing', 'vehicle_light.emergency.not_flashing',
        'vertical_position.off_ground', 'vertical_position.on_ground']
p, r, ap, f1, ap_class, r_attribute, ap_attributes, raw_stats = stats_dataset(nuim_val, '/root/data/nuimages/images/', model, valloader, device, names, attribute_names, nc, conf_thres, iou_thres)

100%|██████████| 4112/4112 [06:22<00:00, 10.74it/s]


[          0           2           3           4           5           6           7           8           9          10          11          12          13          14          15          16          17          18          20          21          22          23          24]
Class                                             	Attribute                               	Recall@0.1
animal                                            	pedestrian.moving                       	0.0588235294
animal                                            	pedestrian.sitting_lying                	0.0000000000
animal                                            	pedestrian.standing                     	0.0217391304
animal                                            	vertical_position.off_ground            	0.0000000000
animal                                            	vertical_position.on_ground             	0.0151515152
human.pedestrian.adult                            	pedestrian.moving                       	0.

In [126]:
print('{:<36}\t{:<36}\t{:<10}'.format('Class', 'Attribute', 'Recall@0.1'))
for i, cls in enumerate(ap_class):
        for j, attri in enumerate(ap_attributes):
                if r_attribute[i, j, 0] != -1:
                        print('{:<36}\t{:<36}\t{:.10f}'.format(names[cls], attribute_names[attri], r_attribute[i, j, 0]))

Class                               	Attribute                           	Recall@0.1
animal                              	pedestrian.moving                   	0.0588235294
animal                              	pedestrian.sitting_lying            	0.0000000000
animal                              	pedestrian.standing                 	0.0217391304
animal                              	vertical_position.off_ground        	0.0000000000
animal                              	vertical_position.on_ground         	0.0151515152
human.pedestrian.adult              	pedestrian.moving                   	0.7969710898
human.pedestrian.adult              	pedestrian.sitting_lying            	0.7012581035
human.pedestrian.adult              	pedestrian.standing                 	0.7436554961
human.pedestrian.child              	pedestrian.moving                   	0.0866454785
human.pedestrian.child              	pedestrian.sitting_lying            	0.1056367655
human.pedestrian.child              	pedestri